In [1]:
chr(0)
print(chr(0))
print("this is a test" + chr(0) + "string")

 
this is a test string


In [19]:
string = "dog.   ??q()$%$$¥"
encode_8 = string.encode('utf-8')
encode_16 = string.encode('utf-16')
print(encode_8)


b'dog.   ??q()$%$$\xc2\xa5'


In [ ]:
import multiprocessing
import time

def square(x):
    """计算平方"""
    return x * x

if __name__ == '__main__':
    # 创建进程池，使用所有可用的CPU核心
    with multiprocessing.Pool() as pool:
        # map方法：将函数应用到序列的每个元素
        results = pool.map(square, range(10))
        print(f"平方结果: {results}")
        
        # apply_async方法：异步执行
        result = pool.apply_async(square, (5,))
        print(f"5的平方: {result.get()}")  # 使用get()获取结果

平方结果: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
5的平方: 25


In [2]:
current_vocab_size = 0
vocab = {}
for i in range(0, 255):
    vocab[current_vocab_size] = bytes([i])
    current_vocab_size += 1
print(vocab)

{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87: b'W', 88: b'X', 89: b'Y', 90: b'Z', 91: b'[',

In [1]:
special_token = "<sqws你>"
special_token = special_token.encode('utf-8')
tokens = [bytes([b]) for b in special_token]


In [2]:
a = [1]
b = [2]
c= a + [b]
print(c)

[1, [2]]


In [5]:
a = (b' t', b'h')
b = (b'i', b'n')

print(a>b)

False


In [8]:


dict_ = {(b' t', b'h'): 2, (b'i', b'n'): 2}

print(max(dict_.items(), key = lambda item:(item[1], item[0]))  )

((b'i', b'n'), 2)


In [10]:
a = b'abc'
b = b'ssabcabcdabc'
print(b.find(a))

2


In [1]:
def find_all_occurrences(main_str, sub_str):
    indexes = []
    start = 0 
    sub_len = len(sub_str)
    main_len = len(main_str)
    
    if sub_len == 0 or sub_len > main_len:
        return indexes
    
    while True:
        pos = main_str.find(sub_str, start)
        if pos == -1:  
            break
        indexes.append(pos)
        start = pos + 1
    return indexes  


main_str = b' here'
sub_str = b're'
print(find_all_occurrences(main_str, sub_str))

[3]


In [ ]:
import os
from typing import BinaryIO, Iterable, List, Tuple
import regex as re
import multiprocessing

def find_chunk_boundaries(
    file: BinaryIO,
    desired_num_chunks: int,
    split_special_token: bytes,
) -> list[int]:
    """
    Chunk the file into parts that can be counted independently.
    May return fewer chunks if the boundaries end up overlapping.
    """
    assert isinstance(split_special_token, bytes), "Must represent special token as a bytestring"

    # Get total file size in bytes
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    file.seek(0)

    chunk_size = file_size // desired_num_chunks

    # Initial guesses for chunk boundary locations, uniformly spaced
    # Chunks start on previous index, don't include last index
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]

    # chunk_count = len(chunk_boundaries) - 1
    chunk_boundaries[-1] = file_size

    mini_chunk_size = 4096  # Read ahead by 4k bytes at a time

    for bi in range(1, len(chunk_boundaries) - 1):
        initial_position = chunk_boundaries[bi]
        file.seek(initial_position)  # Start at boundary guess
        while True:
            mini_chunk = file.read(mini_chunk_size)  # Read a mini chunk

            # If EOF, this boundary should be at the end of the file
            if mini_chunk == b"":
                chunk_boundaries[bi] = file_size
                break

            # Find the special token in the mini chunk
            found_at = mini_chunk.find(split_special_token)
            if found_at != -1:
                chunk_boundaries[bi] = initial_position + found_at
                break
            initial_position += mini_chunk_size

    # Make sure all boundaries are unique, but might be fewer than desired_num_chunks

    return sorted(set(chunk_boundaries))


def pretokenize(chunk: str, special_tokens: list[str]) -> dict[str, int]:
    # Removing special tokens before pre-tokenization
    # This can be done using re.split with "|".join(special_tokens) as the delimiter 
    text_list = re.split("|".join(special_tokens), chunk)
    pattern = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
    pre_tokens = {}
    for chunk in text_list:
        find_words = re.findall(pattern, chunk)
        for word in find_words:
            # word = word.strip()
            pre_tokens[word] = 1 + pre_tokens.get(word, 0)
    return pre_tokens


def train_bpe_tokenizer(
    input_path: str,
    vocab_size: int,
    special_tokens: list[str]
):

    """
    vocab: dict[int, bytes] The tokenizer vocabulary, a mapping from int (token ID in the vocabulary) to bytes (token bytes).
    
    merges: list[tuple[bytes, bytes]] A list of BPE merges produced from training. Each list item
    is a tuple of bytes (<token1>, <token2>), representing that <token1> was merged with
    <token2>. The merges should be ordered by order of creation.
    """
    vocab: dict[int, bytes] = {} # dict[int, bytes]
    merges: list[tuple[bytes, bytes]] = [] 

    token_id = 0
    for i in range(256):
        vocab[token_id] = bytes([i])
        token_id += 1

    for token in special_tokens:
        vocab[token_id] = token.encode("utf-8")
        token_id += 1
    
    num_processes = os.cpu_count()

    pre_token_count: dict[str, int] = {}
    pre_token_list: list[str] = []

    with open(input_path, "rb") as f:
        boundaries = find_chunk_boundaries(f, num_processes, b"<|endoftext|>")
        start_end_pairs = list(zip(boundaries[:-1], boundaries[1:]))
        chunk_list = []
        for start, end in start_end_pairs:
            f.seek(start)
            chunk = f.read(end - start).decode('utf-8', errors='ignore')
            chunk_list.append(chunk)
        num_processes = min(num_processes, len(chunk_list))
        with multiprocessing.Pool(processes=num_processes) as pool:
            results = pool.starmap(pretokenize, [(chunk, special_tokens) for chunk in chunk_list])
            for res in results:
                for token, count in res.items():
                    if token not in pre_token_count:
                        pre_token_list.append(token)
                    pre_token_count[token] = pre_token_count.get(token, 0) + count

    # Initialize token_sequences for each pre-token string
    token_sequences = []
    for pre_token in pre_token_list:
        bytes_data = pre_token.encode('utf-8')
        tokens = [bytes([b]) for b in bytes_data]
        token_sequences.append(tokens)

    # Initialize token_pair_count and token_pair_to_indices
    token_pair_count = {}
    token_pair_to_indices = {}
    for i, tokens in enumerate(token_sequences):
        count_i = pre_token_count[pre_token_list[i]]
        for j in range(len(tokens)-1):
            pair = (tokens[j], tokens[j+1])
            token_pair_count[pair] = token_pair_count.get(pair, 0) + count_i
            if pair not in token_pair_to_indices:
                token_pair_to_indices[pair] = set()
            token_pair_to_indices[pair].add(i)

    # Train BPE tokenizer
    while len(vocab) < vocab_size:
        if not token_pair_count:
            break
        # Find the most frequent token pair
        pair = max(token_pair_count, key=token_pair_count.get)
        count = token_pair_count[pair]
        if count < 2:
            break

        merges.append(pair)
        new_token = pair[0] + pair[1]
        current_id = len(vocab)
        vocab[current_id] = new_token

        # Get all indices that contain this pair
        indices = token_pair_to_indices.get(pair, set()).copy()
        for i in indices:
            tokens = token_sequences[i]
            new_tokens = []
            j = 0
            while j < len(tokens):
                if j < len(tokens)-1 and tokens[j] == pair[0] and tokens[j+1] == pair[1]:
                    new_tokens.append(new_token)
                    j += 2
                else:
                    new_tokens.append(tokens[j])
                    j += 1

            # Update token pairs for this pre-token string
            count_i = pre_token_count[pre_token_list[i]]

            # Remove all old pairs from the original token sequence
            for k in range(len(tokens)-1):
                old_pair = (tokens[k], tokens[k+1])
                if old_pair in token_pair_count:
                    token_pair_count[old_pair] -= count_i
                    if token_pair_count[old_pair] <= 0:
                        del token_pair_count[old_pair]
                if old_pair in token_pair_to_indices:
                    token_pair_to_indices[old_pair].discard(i)
                    if not token_pair_to_indices[old_pair]:
                        del token_pair_to_indices[old_pair]

            # Add all new pairs from the new token sequence
            for k in range(len(new_tokens)-1):
                new_pair = (new_tokens[k], new_tokens[k+1])
                token_pair_count[new_pair] = token_pair_count.get(new_pair, 0) + count_i
                if new_pair not in token_pair_to_indices:
                    token_pair_to_indices[new_pair] = set()
                token_pair_to_indices[new_pair].add(i)

            # Update the token sequence for this index
            token_sequences[i] = new_tokens

        # Remove the merged pair from token_pair_count and token_pair_to_indices
        if pair in token_pair_count:
            del token_pair_count[pair]
        if pair in token_pair_to_indices:
            del token_pair_to_indices[pair]

    return vocab, merges


In [ ]:
        # del token_pair_count[merge_pair]
        # del token_pair_positions[merge_pair]
        # # position is the index of pre_token_list, counts the occurences of merge_pair in the pre_token
        # for pos, _ in merge_positions.items():
        #     pre_token = pre_token_list[pos]
        #     bytes_data = pre_token.encode('utf-8')
        #     count = pre_token_count[pre_token]
        #      # get the occur positions of merge_pair in the pre_token
        #     tokens = [bytes([b]) for b in bytes_data]
        #     occur_pos = find_all_occurrences(bytes_data, new_token)
        #     # del left pair and right pair, add new left pair and new right pair
        #     for start in occur_pos:
        #         if start > 0:
        #             left_token = tokens[start-1]
        #             i = start - 2
        #             while i > 0:
        #                 possible_left_token = tokens[i] + left_token
        #                 if possible_left_token in vocab.values():
        #                     left_token = possible_left_token
        #                     i -= 1
        #                 else:
        #                     break
        #             left_pair = (left_token, merge_pair[0])
        #             if left_pair in token_pair_count:
        #                 token_pair_count[left_pair] -= count
        #                 if token_pair_count[left_pair] <= 0:
        #                     del token_pair_count[left_pair]
        #                     del token_pair_positions[left_pair]
        #                 elif pos in token_pair_positions[left_pair]:
        #                     token_pair_positions[left_pair][pos] -= 1
                            
        #             new_pair = (left_token, new_token)
        #             token_pair_count[new_pair] = token_pair_count.get(new_pair, 0) + count
        #             if new_pair not in token_pair_positions:
        #                 token_pair_positions[new_pair] = {}
        #             token_pair_positions[new_pair][pos] = token_pair_positions[new_pair].get(pos, 0) + 1

        #             # if pre_token == " here" and new_token == b're':
        #             #     print("here........")
        #             #     print('new_pair:', new_pair)
        #             #     print(token_pair_positions[new_pair])

        #         if start + len(merge_pair[0]) + len(merge_pair[1]) < len(tokens):
        #             # right_pair = (merge_pair[1], tokens[start + len(merge_pair[0])+ len(merge_pair[1])])
        #             right_token = tokens[start + len(merge_pair[0]) + len(merge_pair[1])]
        #             i = start + len(merge_pair[0]) + len(merge_pair[1]) + 1
        #             while i < len(tokens):
        #                 possible_right_token = right_token + tokens[i]
        #                 if possible_right_token in vocab.values() and possible_right_token != new_token:
        #                     right_token = possible_right_token
        #                     i += 1
        #                 else:
        #                     break
        #             right_pair = (merge_pair[1], right_token)
        #             if right_pair in token_pair_count:
        #                 token_pair_count[right_pair] -= count
        #                 if token_pair_count[right_pair] <= 0:
        #                     del token_pair_count[right_pair]
        #                     del token_pair_positions[right_pair]
        #                 elif pos in token_pair_positions[right_pair]:
        #                     token_pair_positions[right_pair][pos] -= 1
        #             new_pair = (new_token, right_token)
        #             token_pair_count[new_pair] = token_pair_count.get(new_pair, 0) + count
        #             if new_pair not in token_pair_positions:
        #                 token_pair_positions[new_pair] = {}
        #             token_pair_positions[new_pair][pos] = token_pair_positions[new_pair].get(pos, 0) + 1